In [ ]:
#Esta abordagem é filtragem colaborativa baseada em itens (item-based collaborative filtering)
!pip install pandas numpy scipy scikit-learn

In [ ]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
#Configurar diretório e carregar dados

!wget http://files.grouplens.org/datasets/movielens/ml-latest-small.zip
!unzip ml-latest-small.zip

dataset_dir = "ml-latest-small"

#Carregar avaliações
ratings = pd.read_csv(f"{dataset_dir}/ratings.csv")  #userId, movieId, rating, timestamp

#Carregar filmes
movies = pd.read_csv(f"{dataset_dir}/movies.csv")    #movieId, title, genres

print("Dados carregados!")
print(f"Total de avaliações: {len(ratings)}")
print(f"Total de filmes: {len(movies)}")

--2025-10-19 19:22:34--  http://files.grouplens.org/datasets/movielens/ml-latest-small.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.96.204
Connecting to files.grouplens.org (files.grouplens.org)|128.101.96.204|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://files.grouplens.org/datasets/movielens/ml-latest-small.zip [following]
--2025-10-19 19:22:34--  https://files.grouplens.org/datasets/movielens/ml-latest-small.zip
Connecting to files.grouplens.org (files.grouplens.org)|128.101.96.204|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 978202 (955K) [application/zip]
Saving to: ‘ml-latest-small.zip.1’

ml-latest-small.zip 100%[===================>] 955.28K  2.61MB/s    in 0.4s    

2025-10-19 19:22:35 (2.61 MB/s) - ‘ml-latest-small.zip.1’ saved [978202/978202]

Archive:  ml-latest-small.zip
replace ml-latest-small/links.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: ml-latest-smal

In [23]:
#Mapear IDs para índices e criar matriz esparsa

#O objetivo aqui é transformar os IDs originais (que não são sequenciais)
#em índices contínuos (0, 1, 2, ...) — algo necessário para criar uma matriz.

#Cria dicionários de mapeamento de IDs para índices
#Cada linha em ratings representa uma avaliação feita por um usuário a um filme
#.unique() retorna um array NumPy contendo apenas os valores distintos de uma coluna, na ordem em que aparecem
#enumerate() transforma uma sequência em pares (índice, valor)
'''Se ratings.csv contiver:
   | userId | movieId | rating |
   | ------ | ------- | ------ |
   | 10     | 5       | 3.5    |
   | 42     | 2       | 4.0    |
   | 10     | 7       | 2.0    |
   Então user_mapper = {10: 0, 42: 1}, movie_mapper = {5: 0, 2: 1, 7: 2}
   Usuário 10 vira linha 0, Usuário 42 vira linha 1
   Filme 5 vira coluna 0, Filme 2 vira coluna 1, Filme 7 vira coluna 2
   Matriz esparsa ficará com shape (2, 3) — 2 usuários × 3 filmes.
'''

#Cria mapeamento de cada userId único para um índice sequencial
user_mapper = {uid: i for i, uid in enumerate(ratings['userId'].unique())}
#Cria mapeamento de cada movieId único para um índice sequencial
movie_mapper = {mid: i for i, mid in enumerate(ratings['movieId'].unique())}

#Mapear IDs originais para índices
ratings['user_idx'] = ratings['userId'].map(user_mapper)
ratings['movie_idx'] = ratings['movieId'].map(movie_mapper)

#NOVO: normalização das notas + ponderação por similaridade (vizinho mais próximo)
#Usuários têm padrões de avaliação diferentes: uns dão sempre 5 estrelas, outros quase nunca passam de 3.5
#Ao calcular a similaridade, devemos centralizar (subtrair a média de cada usuário)
#Isso remove o “viés do usuário” e aumenta significativamente a qualidade preditiva do modelo
#O modelo se aproxima mais de um Collaborative Filtering baseado em correlação de Pearson

#Calcular média das notas por usuário
user_mean = ratings.groupby('userId')['rating'].mean()
#Subtrair a média de cada usuário
ratings['rating_norm'] = ratings.apply(lambda x: x['rating'] - user_mean.loc[x['userId']], axis=1)

#Criar matriz esparsa (usuários x filmes)
ratings_sparse = csr_matrix(
    (ratings['rating_norm'], (ratings['user_idx'], ratings['movie_idx']))
)

'''Com o exemplo usado antes:
   user_mapper = {10: 0, 42: 1}
   movie_mapper = {5: 0, 2: 1, 7: 2}
   A matriz de usuários × filmes fica:        5     2     7
                                       U10 [ 3.5   0.0   2.0 ]
                                       U42 [ 0.0   4.0   0.0 ]
   Então a representação esparsa (armazenando só valores != 0) fica:
            (0,0)  3.5 --> user 10, movie 5
            (0,2)  2.0 --> user 10, movie 7
            (1,1)  4.0 --> user 42, movie 2
'''

#Mapeamento inverso (para retornar IDs originais)
movie_inv_mapper = {i: mid for mid, i in movie_mapper.items()}
user_inv_mapper = {i: uid for uid, i in user_mapper.items()}

print("Matriz esparsa criada!")
print(f"Shape da matriz: {ratings_sparse.shape}")

Matriz esparsa criada!
Shape da matriz: (610, 9724)


In [24]:
#Calcular similaridade entre filmes

#.T = operador de transposição. ratings_sparse tem usuários nas linhas e filmes nas colunas
#Transposição é feita para ter filmes nas linhas (necessário para filme-filme similarity)
#dense_output=False mantém saída esparsa.
movie_similarity = cosine_similarity(ratings_sparse.T, dense_output=False)

print("Matriz de similaridade entre filmes calculada!")

Matriz de similaridade entre filmes calculada!


In [25]:
#Função para recomendar filmes similares
#Retorna os top_n filmes mais similares a um movie_id dado
def recommend_movies(movie_id, top_n=10):
    #Obter índice do filme
    if movie_id not in movie_mapper:
        print(f"Filme {movie_id} não encontrado.")
        return []

    #Converte movie_id original para o índice interno da matriz
    movie_idx = movie_mapper[movie_id]

    #Extrai a linha correspondente do filme na matriz de similaridade
    #(cada valor indica o quão parecido é com outro filme)
    sim_scores = movie_similarity[movie_idx].toarray().flatten()

    #Cria uma Series (vetor do pandas) com os scores de similaridade, movie_idx como índice
    sim_series = pd.Series(sim_scores, index=range(len(sim_scores)))

    #Seleciona os índices dos filmes mais similares, em ordem decrescente
    #iloc[1:] ignora o próprio filme (similaridade = 1 consigo mesmo)
    top_indices = sim_series.sort_values(ascending=False).iloc[1:top_n+1].index

    #Monta a lista de recomendações (título e valor da similaridade)
    recommendations = []
    for idx in top_indices:
        mid = movie_inv_mapper[idx]                                   #Converte índice interno de volta para movieId original
        title = movies.loc[movies.movieId == mid, 'title'].values[0]  #Busca título do filme
        score = sim_series[idx]                                       #Similaridade calculada
        recommendations.append((title, score))

    return recommendations

In [26]:
#Testar recomendações para um filme

#Exemplo: movieId = 1 (Toy Story 1995)
movie_id = 1
top_movies = recommend_movies(movie_id, top_n=10)

print(f"Top 10 filmes similares a '{movies.loc[movies.movieId==movie_id,'title'].values[0]}':")
for title, score in top_movies:
    print(f"{title} — similaridade {score:.3f}")

Top 10 filmes similares a 'Toy Story (1995)':
Toy Story 2 (1999) — similaridade 0.403
Toy Story 3 (2010) — similaridade 0.327
Aladdin (1992) — similaridade 0.327
Wallace & Gromit: The Wrong Trousers (1993) — similaridade 0.305
Back to the Future (1985) — similaridade 0.277
Incredibles, The (2004) — similaridade 0.275
Blazing Saddles (1974) — similaridade 0.272
Finding Nemo (2003) — similaridade 0.263
Ghostbusters (a.k.a. Ghost Busters) (1984) — similaridade 0.250
Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981) — similaridade 0.237


In [27]:
#Função para recomendar para um usuário
#Retorna recomendações para um usuário baseado em ratings passados e similaridade filme-filme
def recommend_for_user(user_id, top_n=5):
    #Obter índice do usuário
    if user_id not in user_mapper:
        print(f"Usuário {user_id} não encontrado.")
        return []

    #Converte user_id original para o índice interno da matriz
    user_idx = user_mapper[user_id]

    #Filmes avaliados pelo usuário (linha do usuário na matriz)
    #Converte a linha esparsa para array e achata para vetor 1D
    user_ratings = ratings_sparse[user_idx, :].toarray().flatten()

    #Pega os índices dos filmes avaliados (notas > 0)
    rated_indices = np.where(user_ratings > 0)[0]

    #Vetor inicial para acumular scores de recomendação
    #(um valor predito para cada filme)
    scores = np.zeros(ratings_sparse.shape[1])

    '''Para cada filme que o usuário avaliou, acumular score ponderado:
       - pega as similaridades desse filme com todos os outros
       - soma uma contribuição proporcional à nota que o usuário deu'''
    for idx in rated_indices:
        sim_scores = movie_similarity[idx].toarray().flatten()
        scores += sim_scores * user_ratings[idx]

    #NOVO: Normalizar pelas somas de similaridades positivas
    norm_factor = np.array(movie_similarity[rated_indices, :].sum(axis=0)).flatten()
    norm_factor[norm_factor == 0] = 1
    scores = scores / norm_factor

    #Zera os scores dos filmes já avaliados para não recomendar algo que o usuário já viu
    scores[rated_indices] = 0

    #Seleciona os índices dos filmes com maiores scores preditos
    top_indices = np.argsort(scores)[::-1][:top_n]

    #Monta a lista final de recomendações com títulos e pontuações
    recommendations = []
    for idx in top_indices:
        mid = movie_inv_mapper[idx]   #Converte índice interno para movieId original
        title = movies.loc[movies.movieId == mid, 'title'].values[0]
        recommendations.append((title, scores[idx]))

    return recommendations

In [28]:
#Testar recomendações para um usuário

user_id = 18
top_user_movies = recommend_for_user(user_id, top_n=5)

print(f"Top 5 recomendações para o usuário {user_id}:")
for title, score in top_user_movies:
    print(f"{title} — score {score:.3f}")

Top 5 recomendações para o usuário 18:
Kung Fu Panda: Secrets of the Masters (2011) — score 18.460
Bent (1997) — score 15.307
Boiling Point (1993) — score 12.145
13th (2016) — score 11.569
Stop Making Sense (1984) — score 10.321
